In [1]:
import pandas as pd
from numpy.linalg import norm
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.metrics import mean_squared_error,recall_score,ndcg_score
from sklearn.model_selection import ShuffleSplit

In [3]:
'''
np.corrcoef
Return Pearson product-moment correlation coefficients.
'''

'\nnp.corrcoef\nReturn Pearson product-moment correlation coefficients.\n'

In [2]:
def Average(lst): 
	return round(sum(lst) / len(lst),4) 

In [3]:
user_movie = np.load('../hw3_data/Movielens/user_movie.npy')

X = user_movie[:,:2]
y = user_movie[:,-1]
entity_user = 943
entity_item = 1682
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rs = ShuffleSplit(n_splits=5, test_size=.1, random_state=42)


In [13]:
user_business = np.load('../hw3_data/Yelp/user_business.npy')
X = user_business[:,:2]
y = user_business[:,-1]
entity_user = 16239
entity_item = 14284
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rs = ShuffleSplit(n_splits=5, test_size=.1, random_state=42)


In [90]:
user_book = np.load('../hw3_data/Douban_Book/user_book.npy')

X = user_book[:,:2]
y = user_book[:,-1]
entity_user = 13024
entity_item = 22347
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rs = ShuffleSplit(n_splits=5, test_size=.1, random_state=42)

In [101]:
pred

nan

In [14]:
#User-based CF
RMSE_M = []
recall_M = []
NDCG_M = []
for i, (train_index, test_index) in enumerate(rs.split(X)):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]

    item_rating = np.zeros((entity_item, entity_user), dtype=int)
    for i in range(y_train.shape[0]):
        user, item = X_train[i]
        item_rating[item-1, user-1] = y_train[i]
        
    #ICF = np.corrcoef(item_rating)
    ICF = cosine_similarity(item_rating)

    Mu = item_rating.sum()/np.count_nonzero(item_rating)
    dev_user, dev_item = np.std(item_rating,axis=0), np.std(item_rating,axis=1)

    predict = []
    for i in range(X_test.shape[0]):

        user, item =  X_test[i]
        print(user, item)
        most_similar_item = np.argpartition(ICF[item-1],-6)[-6:-1]
        msi_weight = ICF[item-1][most_similar_item]
        rank = item_rating[most_similar_item, user-1]

        bu = dev_user[user-1]-Mu
        bi = dev_item[item-1]-Mu
        bj = dev_item[most_similar_item]-Mu
        bui = Mu + bu + bi
        buj = Mu + bu + bj
        weight_rank = (rank-buj)*msi_weight
        pred = bui+ weight_rank[~np.isnan(weight_rank)].sum()/msi_weight[~np.isnan(msi_weight)].sum()
        predict.append(pred)
    
    RMSE = mean_squared_error(np.array(predict),y_test, squared=False)
    RMSE_M.append(RMSE)
    recall = recall_score(np.round(np.array(predict)),y_test, average='micro')
    recall_M.append(recall)
    NDCG = ndcg_score(np.array([predict]),y_test.reshape(1, -1), k=10)
    NDCG_M.append(NDCG) 
    print(f"RMSE: {RMSE}, Recall: {recall}, NDCG: {NDCG}")
print(f"Average RMSE: {Average(RMSE_M)}, Average Recall: {Average(recall_M)}, Average NDCG: {Average(NDCG_M)}")    


10427 13313
15557 12747
15298 1303
14351 2229
4737 3040
13124 10973
1072 14191
958 1950
7690 4409
16057 9106
859 11232
12350 13943
6791 4405
591 10940
12279 7290
1530 11258
4740 5349
12946 4014
15610 2858
5363 10627
11112 9527
4103 10734
3342 8832
14634 7479
14969 5786
15742 4021
564 3572
9376 836
13839 11979
15698 8489
5679 1943
7611 6182
5349 10403
3393 5350
10666 13975
9100 11498
15294 3233
12415 6889
15557 12714
13373 4790
816 6344
15397 2484
4996 10533
12162 2746
11971 9853
954 3816
1888 2033
12476 636
10129 8991
15757 9212
6874 2439
13966 1660
1211 11625
2926 436
9647 657
14538 9347
8343 469
2680 10209
10410 4508
3133 3082
453 5723
6927 11198
13565 10512
8080 6554
13896 1403
4786 3439
8359 7424
9263 5664
1923 4960
7023 789
8705 5219
2230 10103
4538 3512
12099 942
6300 6080
1186 6998
3503 7155
13297 5425
2499 528
1429 9050
3203 3068
7286 8711
6554 10534
11970 3294
5755 7437
14202 6575
9951 12435
10601 9756
2293 5295
15467 3018
6587 938
15237 3567
15672 6668
2673 12601
37 10799
686

/tmp/ipykernel_3467424/3169672721.py:35: RuntimeWarning: invalid value encountered in double_scalars
  pred = bui+ weight_rank[~np.isnan(weight_rank)].sum()/msi_weight[~np.isnan(msi_weight)].sum()


12186 515
8122 5755
11969 8538
1083 7802
5349 12937
13350 12984
13791 10604
4875 5554
7743 1741
6870 2219
11564 10078
2834 11453
6524 11644
10817 2596
8819 13480
13867 1988
8163 13256
621 5507
11972 336
12566 1078
7099 5245
2840 5395
15642 9535
8143 6431
12588 6902
12161 1585
12412 783
5771 4221
6079 2442
3634 3567
14090 10759
3236 896
14948 9102
2184 3156
2154 7473
8089 863
2550 6915
10678 3070
3651 6004
557 5673
3434 4820
12899 7966
8801 7217
15397 4453
2172 7143
13719 1147
5953 11369
7079 3439
950 9480
3553 9880
4278 2675
1186 2067
5593 12178
9154 2540
9497 6837
37 3299
5232 388
3787 7422
1644 2661
11532 7935
809 11020
3233 14121
7982 1577
2180 12734
12899 9260
7865 7519
15503 3572
12160 9396
12088 3880
14542 13094
9780 3420
3307 9529
716 5301
11816 11129
10539 6004
2882 1763
416 9529
6527 1094
9904 10535
564 12410
14124 12262
1643 1563
6257 9298
1441 11345
4190 9244
3088 1823
9780 4210
3787 8606
10145 11463
12418 2694
11747 12838
12160 1502
11279 6803
15520 9788
5515 13925
10849 63

ValueError: Input contains NaN.

In [ ]:
# 扣除所有使用者的加總平均，undo
mean_by_user = user_rating.sum(axis=1)/np.count_nonzero(user_rating, axis=1)
mean_by_user
none_zero = np.where(user_rating > 0)
none_zero[]

In [ ]:
UFC = np.load('cs_user_rating.npy')
UFC

In [10]:
np.isnan(predict).sum()

14

In [132]:
rank = item_rating[most_similar_item, user-1]
rank

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [141]:
san = ((rank-buj)*msi_weight)

In [142]:
weight_rank[~np.isnan(weight_rank)]

array([0.45880565, 1.01543139])

In [145]:

bu = dev_user[user-1]-Mu
bi = dev_item[item-1]-Mu
bj = dev_item[most_similar_item]-Mu
bui = Mu + bu + bi
buj = Mu + bu + bj
weight_rank = (rank-buj)*msi_weight


In [148]:
weight_rank = (rank-buj)*msi_weight


In [150]:
msi_weight[~np.isnan(msi_weight)].sum()

1.367763603682886

In [151]:
pred = bui+ weight_rank[~np.isnan(weight_rank)].sum()/msi_weight[~np.isnan(msi_weight)].sum()

In [152]:
pred = bui+ weight_rank[~np.isnan(weight_rank)].sum()/msi_weight[~np.isnan(msi_weight)].sum()
pred

0.062413521779772996

In [120]:
np.argpartition(ICF[item-1],-6)[-61:-1]
ICF[item-1][most_similar_item]

array([nan, nan, nan, nan, nan])

In [114]:
np.count_nonzero(~np.isnan(ICF))

2782224

In [118]:
entity_item*entity_item

2829124

In [115]:
item_rating[most_similar_item, user-1]

array([0, 0, 0, 0, 0])

In [105]:
msi_weight = ICF[item-1][most_similar_item]

rank = item_rating[most_similar_item, user-1]
msi_weight,rank

(array([nan, nan, nan, nan, nan]), array([0, 0, 0, 0, 0]))

In [ ]:
ICF[item-1]

In [ ]:


bu = dev_user[user-1]-Mu
bi = dev_item[item-1]-Mu
bj = dev_item[most_similar_item]-Mu
bui = Mu + bu + bi
buj = Mu + bu + bj

pred = bui+((rank-buj)*msi_weight).sum()/msi_weight.sum()


# try

In [6]:
for i, (train_index, test_index) in enumerate(rs.split(X)):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]

In [7]:
X_train

array([[ 2970,  4514],
       [  370,   153],
       [10913,  1939],
       ...,
       [ 5744,  1910],
       [10926,  6956],
       [ 4962,   151]])

In [8]:
item_rating = np.zeros((22347, 13024), dtype=int)
for i in range(y_train.shape[0]):
    user, item = X_train[i]
    item_rating[item-1, user-1] = y_train[i]

ICF = cosine_similarity(item_rating)

user, item =  X_test[0]
most_similar_item = np.argpartition(ICF[item-1],-6)[-6:-1]
most_similar_item


In [30]:
predict = []
for i in range(X_test.shape[0]):
    user, item =  X_test[i]
    most_similar_item = np.argpartition(ICF[item-1],-6)[-6:-1]
    msi_weight = ICF[item-1][most_similar_item]
    rank = item_rating[most_similar_item,item-1]
    pred = (rank*msi_weight).sum()/msi_weight.sum()
    predict.append(pred)

array([ 340,  776, 1970, 1968, 2692])

In [61]:
user, item =  X_test[0]
most_similar_item = np.argpartition(ICF[item-1],-6)[-6:-1]
msi_weight = ICF[item-1][most_similar_item]

rank = item_rating[most_similar_item,item-1]
pred = (rank*msi_weight).sum()/msi_weight.sum()


(array([ 340,  776, 1970, 1968, 2692]),
 array([0.19357475, 0.2017085 , 0.27383471, 0.21984672, 0.30852758]))

In [55]:
Mu = item_rating.sum()/np.count_nonzero(item_rating)
dev_user, dev_item = np.std(item_rating,axis=0), np.std(item_rating,axis=1)


In [70]:

bu = dev_user[user-1]-Mu
bi = dev_item[item-1]-Mu
bj = dev_item[most_similar_item]-Mu

bui = Mu + bu + bi
buj = Mu + bu + bj


In [72]:

((rank-buj)*msi_weight).sum()/msi_weight.sum()

3.545993156184112

In [69]:
(rank-bj)

array([3.48486829, 3.65443163, 3.79985759, 3.77380215, 3.8199192 ])

In [60]:
most_similar_item

array([ 340,  776, 1970, 1968, 2692])

In [59]:
bui

-3.407117189742817

In [ ]:
# 扣除所有使用者的加總平均，undo
mean_by_user = user_rating.sum(axis=1)/np.count_nonzero(user_rating, axis=1)
mean_by_user
none_zero = np.where(user_rating > 0)
none_zero[]